# Lab 4. Energy Agent Collaborator

## はじめに

前回のノートブックでは、Amazon Bedrock でマルチエージェント コラボレーター機能を作成する方法を説明しました。

[マルチエージェント コラボレーション](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-multi-agents-collaboration.html) は、エージェント間の階層的なコラボレーションを可能にする Amazon Bedrock Agents のネイティブ機能です。これで、エージェント コラボレーションを有効にして、セカンダリ エージェントをスーパーバイザー エージェントに関連付けることができます。これらのセカンダリ エージェントは、同じアカウント内の既存のエージェントであればどれでもかまいません (コラボレーションを持つエージェントも含む)。

このノートブックでは、前回のラボで作成したスーパーバイザー エージェントとはやり取りしません。

以下は、マルチエージェント コラボレーターの完全なアーキテクチャを表しています。

![アーキテクチャ](img/energy_manager_agent.png)

このラボでは、詳細なトレースを使用して一連のテストを実行し、エージェントで行われている思考の連鎖の推論を確認します。

**[思考の連鎖の推論](https://docs.aws.amazon.com/bedrock/latest/userguide/what-is-prompt-engineering.html)** は、プロンプトに基づいて回答がどのように導き出されるかを段階的に推論するプロンプトエンジニアリング手法です。

Amazon Bedrock Agents では、[トレース](https://docs.aws.amazon.com/bedrock/latest/userguide/trace-events.html) を使用して、エージェントの段階的な推論プロセスを理解できます。マルチエージェントコラボレーション機能を使用すると、スーパーバイザートレースはサブエージェントの推論の詳細も提供します。

## セットアップ

boto3 のバージョンが最新であることを確認してください。

そうでない場合は、[notebook 1](../1-energy-forecast/1_forecasting_agent.ipynb) を返さず、セットアップ ブロックを再度実行してください。

In [ ]:
!pip freeze | grep boto3

パッケージを有効にするにはカーネルを再起動してください

In [ ]:
import IPython

# IPython.Application.instance().kernel.do_shutdown(True)

boto3のバージョンが最新かどうかを確認してください

In [ ]:
!pip freeze | grep boto3

## マルチエージェント コラボレーターの呼び出し

異なるクエリを使用してスーパーバイザー エージェントを呼び出し、異なるサブエージェントの呼び出しをトリガーしてみましょう。その前に、`bedrock_agent_helper.py` を Python パスに追加して、ファイルが認識され、呼び出されるようにします。

次に、ヘルパー クラス `bedrock_agent_helper.py` をインポートします。

これらのファイルには、ラボをスムーズに実行することに重点を置いたヘルパー クラスが含まれています。

Bedrock とのすべてのやり取りは、これらのクラスによって処理されます。

このラボで呼び出すメソッドは次のとおりです。

`agents.py` の場合:

- `invoke`: クエリを使用してエージェントを呼び出す

In [ ]:
import sys
import uuid
import time

sys.path.insert(0, ".")
sys.path.insert(1, "..")

from utils.bedrock_agent_helper import (
    AgentsForAmazonBedrock
)
agents = AgentsForAmazonBedrock()

続行する前に、環境内のスーパーバイザー エージェントとそのサブエージェントから情報を回復しましょう。

In [ ]:
%store -r
print("Supervisor agent name:", energy_agent_name, "id:", energy_agent_id)
print("Forecast agent name:", forecast_agent_name, "id:", forecast_agent_id, "alias_id:", forecast_agent_alias_id)
print("Solar Panel agent name:", solar_agent_name, "id:", solar_agent_id, "alias_id:", solar_agent_alias_id)
print("Peak Management agent name:", peak_agent_name, "id:", peak_agent_id, "alias_id:", peak_agent_alias_id)

トレースをより良く視覚化するために、エージェント名を含む辞書を設定できるようになりました。

In [ ]:
multi_agent_names = {
    f"{forecast_agent_id}/{forecast_agent_alias_id}": forecast_agent_name,
    f"{solar_agent_id}/{solar_agent_alias_id}": solar_agent_name,
    f"{peak_agent_id}/{peak_agent_alias_id}": peak_agent_name,
    f"{energy_agent_id}/{energy_agent_alias_id}": energy_agent_name
}

次のセクションでは、マルチエージェントコラボレーターによってサブエージェントを呼び出します。

### ラボ 1 - 予測エージェント
まず、予測エージェントを要求するプロンプトを使用してエージェントにクエリを実行します。

In [ ]:
%%time
session_id:str = str(uuid.uuid1())

response = agents.invoke(
    "Can you give me my forecasted energy consumption? How does it compare to my current usage? My id is 1", 
    energy_agent_id,
    session_id=session_id,
    enable_trace=True,
    multi_agent_names=multi_agent_names
)
print("====================")
print(response)

In [ ]:
time.sleep(60)

In [ ]:
%%time
session_id:str = str(uuid.uuid1())

response = agents.invoke(
    "can you give me my past energy consumption? What is my average spending on summer months? My customer id is 1", 
    energy_agent_id,
    session_id=session_id,
    enable_trace=True,
    multi_agent_names=multi_agent_names
)
print("====================")
print(response)

In [ ]:
time.sleep(60)

In [ ]:
%%time
session_id:str = str(uuid.uuid1())

response = agents.invoke(
    "Can you update my forecast for month 12/2024? I will be travelling and my estimate will be 70. My id is 1", 
    energy_agent_id,
    session_id=session_id,
    enable_trace=True,
    multi_agent_names=multi_agent_names
)
print("====================")
print(response)

In [ ]:
time.sleep(60)

In [ ]:
%%time
session_id:str = str(uuid.uuid1())

response = agents.invoke(
    "Can you give me my forecasted energy consumption month by month? My id is 1", 
    energy_agent_id,
    session_id=session_id,
    enable_trace=True,
    multi_agent_names=multi_agent_names
)
print("====================")
print(response)

In [ ]:
time.sleep(60)

#### ラボ 2 - ソーラーパネルエージェント

In [ ]:
%%time
session_id:str = str(uuid.uuid1())
response = agents.invoke(
    "how can I check if my Sunpower double-X solar panel eletrical consumption is compliant with energy rules?", 
    energy_agent_id,
    session_id=session_id,
    enable_trace=True
)
print("====================")
print(response)

In [ ]:
time.sleep(60)

In [ ]:
%%time
session_id:str = str(uuid.uuid1())

response = agents.invoke(
    "Can I get all tickets that I have? My customer id is 1", 
    energy_agent_id,
    session_id=session_id,
    enable_trace=True,
    multi_agent_names=multi_agent_names
)
print("====================")
print(response)

In [ ]:
time.sleep(60)

#### ラボ 3 - ピーク負荷マネージャーエージェント

In [ ]:
%%time
session_id:str = str(uuid.uuid1())

response = agents.invoke(
    "What's causing my peak load? My id is 2.", 
    energy_agent_id,
    session_id=session_id,
    enable_trace=True,
    multi_agent_names=multi_agent_names
)
print("====================")
print(response)

In [ ]:
time.sleep(60)

In [ ]:
%%time
session_id:str = str(uuid.uuid1())

response = agents.invoke(
    "Is it possible to optimize my consumption? My id is 1", 
    energy_agent_id,
    session_id=session_id,
    enable_trace=True,
    multi_agent_names=multi_agent_names
)
print("====================")
print(response)

### 次のステップ

おめでとうございます。ワークショップは完了です。

これまでに、3 つのサブエージェントと 1 つのスーパーバイザー エージェントを作成しました。複数のサブエージェントを必要とするプロンプトを使用してスーパーバイザー エージェントを呼び出しました。

その他のマルチエージェント コラボレーションの例については、[Amazon Bedrock エージェント サンプル](https://github.com/awslabs/amazon-bedrock-agent-samples) リポジトリを確認してください。

次に、予期しないコストを回避するためにリソースをクリーンアップしましょう。